## How does PyNN construct multicompartment morphology?

### 1. The construction is built upon [NeuroML](https://neuroml.org/neuromlv2)
Specifically, PyNN uses three classes available in [pyNeuroML](https://pypi.org/project/pyNeuroML/)

* `Point3DWithDiam`
   - a function that takes creates a 3D point given its x-, y-, z-coordinate and its diameter (in um)
* `Segment`
   - a function that creates a NeuroML-segment whose ends are given by its proximal and distal 3D point.
   - NeuroML-segment is not exactly the same as a segment in NEURON or Arbor.
     [Here](https://github.com/NeuroML/NeuroML2/blob/master/examples/NML2_FullCell.nml) is an example of NeuroML-segment.
* `Morphology`
   - a function that construct a morphology for a given number of segments

**NOTE:**
[NeuroML](https://neuroml.org/neuromlv2) and [pyNeuroML](https://github.com/NeuroML/pyNeuroML) lacks the documentation for the functions available when using `import neuroml`, let alone the documentation for a `neuroml` function. Also, the [github source code](https://github.com/NeuroML/pyNeuroML) is opaque for a common user.

For looking at the how these function work one may use a built-in module of Python, `import inspect` and passing the neuroml class/function in question as argument like,
```
inspect.getmodule( Morphology )
```
one can identify the working source code.

The above three `neuroml` classes are all in the script
`~/envs/<env-name>/lib/python3.8/site-packages/neuroml/nml/nml.py`

In [1]:
from neuroml import Morphology, Segment, Point3DWithDiam as P

In [2]:
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma", id=0)
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma, id=1)

### 2. How is the NeuroML morphology made compatible with PyNN?
PyNN does not directly utilize the NeuroML morphology (output of `neuroml.Morphology`). Instead, PyNN has an interface that takes the NeuroML morphology such that it creates a morphology that is suitable for use by [`pyNN.<sim>.MultiCompartmentNeuron`](https://github.com/myHBPwork/PyNN/blob/mc/pyNN/neuron/standardmodels/cells.py)(which in turn is a child of [class of same name](https://github.com/myHBPwork/PyNN/blob/mc/pyNN/standardmodels/cells.py).

This interface is called `NeuroMLMorphology`.

In [3]:
from pyNN.morphology import NeuroMLMorphology, uniform

In [4]:
morph = NeuroMLMorphology(Morphology(segments=(soma, dend)))

### 2.1. What are the methods available in the instance of `NeuroMLMorphology`?
Looking at the output of `help(morph)` these are
- `get_diameter`
- `get_distance`
- `id_map`
- `labels`
- `path_lengths`
- `section_groups`
- `segments`
- `soma_index`
- `synamptic_receptors`

For the morphology we created above `morph.segments` returns the tuple of segments in the given morphology

In [6]:
morph.segments

(<Segment|0|soma>, <Segment|1|dendrite>)

### 2.2. What are the available attributes for a given segment?
For the first segment in our morphology (`morph.segments[0]`) the available attributes can be identified either with `help(morph.segments[0])` but it is clearer with `dir(morph.segments[0])`, which returns the attributes as a list.

Some of these attributes are:
* `morph.segments[0].proximal` and `morph.segments[0].distal`
  - this returns the diameter and its coordinates of the respective 3D point, proximal or distal.
* `morph.segments[0].parent`
  - returns its parent segment if it exist (otherwise it returns nothing)
  - when the parent segment does not exist this is equivalent to
  ```
  >>> not morph.segments[0].parent
  True
  ```
* `morph.segments[0].name`
  - returns the name of the segment
* `morph.segments[0].id`
  - returns the NeuroML-segment id integer number
  - Note that this should not be confused with integer for structure identification like those in [SWC](http://www.neuronland.org/NLMorphologyConverter/MorphologyFormats/SWC/Spec.html)

In [19]:
morph.segments[0].proximal

(0.0, 0.0, 0.0), diam 18.8um

In [38]:
not morph.segments[0].parent

True

In [28]:
morph.segments[1].parent

<Segment|0|soma>

In [41]:
morph.segments[0].name

'soma'

In [39]:
morph.segments[0].id

0

#### 2.2.1 Extracting the parameters of the proximal and distal points of a segment.
As mentioned above, although `morph.segments[0].proximal` (or `morph.segments[0].distal`) returns the diameter and its coordinates of the respective 3D point(proximal or distal), to get their values one must invoke the respective attributes.

The available attributes for a given point of a segment may be identified using `dir(morph.segments[0].proximal)`. Therefore,
* `morph.segments[0].proximal.x`
  - returns the value of the x-coordinate
* `morph.segments[0].proximal.y`
  - returns the value of the y-coordinate
* `morph.segments[0].proximal.z`
  - returns the value of the z-coordinate
* `morph.segments[0].proximal.diameter`
  - returns the value of the diameter (in um)

In [6]:
[morph.segments[0].proximal.x, morph.segments[0].proximal.y, morph.segments[0].proximal.z,
morph.segments[0].proximal.diameter]

[0.0, 0.0, 0.0, 18.8]

In [33]:
import arbor

In [34]:
from arbor import mpoint
from arbor import mnpos

In [35]:
tree = arbor.segment_tree()

##### Create a segment attached to root

In [37]:
# Start with a cylinder segment for the soma (with tag 1)
tree.append(mnpos, mpoint(0,   0.0, 0, 2.0), mpoint( 4,  0.0, 0, 2.0), tag=1) # `0` segment
#tree.append(mnpos, mpoint(0,   0.0, 0, 2.0), mpoint( 4,  0.0, 0, 2.0)) # does it need to be tagged? YES

0

##### Create a section
NOTE that section => same tag

In [12]:
# Construct the first section of the dendritic tree,
# comprised of segments 1 and 2, attached to soma segment 0.
tree.append(0,     mpoint(4,   0.0, 0, 0.8), mpoint( 8,  0.0, 0, 0.8), tag=3) # `1` segment
tree.append(1,     mpoint(8,   0.0, 0, 0.8), mpoint(12, -0.5, 0, 0.8), tag=3) # `2` segment
# Construct the rest of the dendritic tree.
tree.append(2,     mpoint(12, -0.5, 0, 0.8), mpoint(20,  4.0, 0, 0.4), tag=3) # `3` segment
tree.append(3,     mpoint(20,  4.0, 0, 0.4), mpoint(26,  6.0, 0, 0.2), tag=3) # `4` segment
tree.append(2,     mpoint(12, -0.5, 0, 0.5), mpoint(19, -3.0, 0, 0.5), tag=3) # `5` segment
tree.append(5,     mpoint(19, -3.0, 0, 0.5), mpoint(24, -7.0, 0, 0.2), tag=3) # `6` segment
tree.append(5,     mpoint(19, -3.0, 0, 0.5), mpoint(23, -1.0, 0, 0.2), tag=3) # `7` segment
tree.append(7,     mpoint(23, -1.0, 0, 0.2), mpoint(26, -2.0, 0, 0.2), tag=3) # `8` segment

8

##### Create another section

In [13]:
# Two segments that define the axon, with the first at the root, where its proximal
# end will be connected with the proximal end of the soma segment.
tree.append(mnpos, mpoint(0,   0.0, 0, 2.0), mpoint(-7,  0.0, 0, 0.4), tag=2) # `9` segment
tree.append(9,     mpoint(-7,  0.0, 0, 0.4), mpoint(-10, 0.0, 0, 0.4), tag=2) # `10` segment

10

In [15]:
morph = arbor.morphology(tree)

In [19]:
help(arbor.morphology)

Help on class morphology in module arbor._arbor:

class morphology(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      morphology
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: arbor._arbor.morphology, arg0: arbor._arbor.segment_tree) -> None
 |  
 |  __str__(...)
 |      __str__(self: arbor._arbor.morphology) -> str
 |  
 |  branch_children(...)
 |      branch_children(self: arbor._arbor.morphology, i: int) -> List[int]
 |      
 |      The child branches of branch i.
 |  
 |  branch_parent(...)
 |      branch_parent(self: arbor._arbor.morphology, i: int) -> int
 |      
 |      The parent branch of branch i.
 |  
 |  branch_segments(...)
 |      branch_segments(self: arbor._arbor.morphology, i: int) -> List[arbor._arbor.msegment]
 |      
 |      A list of the segments in branch i, ordered from proximal to distal ends of the branch.
 |  
 |  ----------------------------

In [20]:
morph.num_branches

6

In [27]:
morph.branch_parent(1)

0

In [28]:
morph.branch_children(1)

[]

In [29]:
morph.branch_segments(1)